## Quickstart

The goal of this software is to enable approximate inference of the phylogenetic history of a distributed digital population solely through analysis of heritable genome annotations.
Put another way, given a scenario where packets of data are being copied and moved within a distributed system, this software enables estimation of how closely any two data packets are related.
More precisely, for any two extant data packets, estimation bounds can be produced for the number of copies elapsed from those packets’ last shared source copy (i.e., most recent common ancestor a.k.a. MRCA) to yield each extant packet.
This is done by means of annotations on the data being copied itself — no centralized tracking system required.

This capability has direct applications in digital evolution research (e.g., artificial life, genetic programming, genetic algorithms), and also may prove useful for other distributed systems applications.

### Installation

In [ ]:
%%capture
!python3 -m pip install hstrat

### Getting Started

In order to track the data (referred to as *stratum*) deposited at each generation, we can use an annotation called a *hereditary stratigraphic column*.


In [ ]:
from hstrat import hstrat

founder1 = hstrat.HereditaryStratigraphicColumn(
    stratum_retention_policy=hstrat.fixed_resolution_algo.Policy(1)
)
founder2 = hstrat.HereditaryStratigraphicColumn(
    stratum_retention_policy=hstrat.fixed_resolution_algo.Policy(3),
)

For each annotation, a specific *stratum retention policy* can be chosen.
Simply put, these are various methods use follow a specific algorithm to "prune", or systematically delete, data in the column to avoid a linear space complexity.
As a downside, pruning results in uncertainty in MRCA generation estimates.

In the example above, the algorithm `fixed_resolution_algo` was chosen.
The specific policy instance is supplied an integer that dictates the amount of pruning, with smaller values resulting in denser stratum retention.
For example, a policy using the fixed resolution algorithm supplied with the value three would retain strata from every third generation.

Below is a quick overview of the five available algorithms.
For a more in depth overview, visit [Choosing a Retention Policy](./policies.html).


| Stratum Retention Algorithm               | Space Complexity | MRCA Gen Uncertainty |
| ----------------------------------------- | ---------------- | -------------------- |
| Fixed Resolution Algorithm                | `n/k`            | `k`                  |
| Recency-proportional Resolution Algorithm | `k * log(n)`     | `m/k`                |
| Depth-proportional Resolution Algorithm   | `k`              | `n/k`                |
| Geometric Sequence Nth Root Algorithm     | `k`              | `m * n^(1/k)`        |
| Curbed Recency-proportional Resolution Algorithm | `k`     | `m / k` -> `m * n^(1/k)` |


### Estimating MCRA
To check for common ancestors between two columns:


In [ ]:
hstrat.does_have_any_common_ancestor(founder1, founder2)

In order for two columns to have a common ancestor, one of the columns must be created from another.
This can be accomplished using the `Clone()` method. The `CloneDescendant()` and `CloneNthDescendant(n)` methods are also available, cloning and then depositing one and n descendants respectively.

In [ ]:
descendant2a = founder2.Clone()
for __ in range(10):
    descendant2a.DepositStratum()

descendant2b = descendant2a.CloneDescendant()
descendant2c = descendant2a.CloneNthDescendant(20)

hstrat.does_have_any_common_ancestor(founder2, descendant2a)

A new generation of stratum can be added by running `DepositStratum()` on specific hereditary stratigraphic columns.
To elapse multiple generations, `DepositStrata()` can also be used, taking in an integer to specify the number of generations elapsed.

If you want to find the number of generations elapsed since the MRCA, `calc_ranks_since_mrca_bounds_with` will return a tuple with the estimated lower and upper bound.
The argument `prior` represents the prior probability density distribution over possible generations of the MRCA.

In [ ]:
hstrat.calc_ranks_since_mrca_bounds_with(
    descendant2b,
    descendant2c,
    prior="arbitrary",
)

In [ ]:
hstrat.calc_ranks_since_mrca_bounds_with(
    descendant2c,
    descendant2b,
    prior="arbitrary",
)

In [ ]:
hstrat.calc_rank_of_mrca_bounds_between(
    descendant2b, descendant2c, prior="arbitrary"
)

### Creating and Using Populations

A genome can be defined with a user-created class, with a specific attribute containing the annotation for each instance.
Using this class, a population of the genome can be created by instantiating a list of objects.

An easier way to generate a population of hereditary stratigraphic columns is from a biopython tree using `descend_template_phylogeny_biopython`.

In [ ]:
import random

random.seed(1)
import dendropy as dp

tree_url = "https://raw.githubusercontent.com/mmore500/hstrat/5069db7c358ac6949ceda5fe8cc9989d5d7139f9/examples/assets/example.newick"
template_tree = dp.Tree.get(url=tree_url, schema="newick")
for node in template_tree:
    node.edge_length = random.randint(1, 10)
template_tree.is_rooted = True
template_tree.ladderize()
print(template_tree.as_ascii_plot(plot_metric="length", width=50))

In [ ]:
extant_population = hstrat.descend_template_phylogeny_dendropy(
    template_tree,
    seed_column=hstrat.HereditaryStratigraphicColumn(
        hstrat.recency_proportional_resolution_algo.Policy(5)
    ),
    extant_nodes=template_tree.leaf_node_iter(),
)

This list can be used to test tree reconstruction on, similar to the results of an actual phlyogenetic simulation.
The function `build_tree` takes in a population and returns a tree constructed in alife standard format.
The `version_pin` parameter dictates how calls to the function should resolve in future releases, with `hstrat.__version__` automatically tracking new updates.


In [ ]:
estimated_phylogeny = hstrat.build_tree(
    extant_population,
    version_pin=hstrat.__version__,
    taxon_labels=map(lambda n: n.taxon.label, template_tree.leaf_node_iter()),
)
estimated_phylogeny

In [ ]:
import alifedata_phyloinformatics_convert as apc

dendropy_tree = apc.alife_dataframe_to_dendropy_tree(
    estimated_phylogeny,
    setup_edge_lengths=True,
)
dendropy_tree.is_rooted = True
dendropy_tree.ladderize()

# draw the reconstruction!
print(dendropy_tree.as_ascii_plot(plot_metric="length", width=50))

### Reading and Displaying Columns

An ascii representation of a hereditary stratigraph column can be printed using the `hstrat.col_to_ascii()` function, passing in the specified column as a parameter.

In [ ]:
col = hstrat.HereditaryStratigraphicColumn(
    stratum_retention_policy=hstrat.fixed_resolution_algo.Policy(3),
)

for __ in range(5):
    col.DepositStratum()

print(hstrat.col_to_ascii(col))

A column can also be exported to pandas dataframe.


In [ ]:
policy = hstrat.recency_proportional_resolution_algo.Policy(3)

col = hstrat.HereditaryStratigraphicColumn(
    stratum_retention_policy=policy,
)
for __ in range(100):
    col.DepositStratum()

hstrat.col_to_dataframe(col)

### Further Reading
- [Choosing a Retention Policy](./policies.html)
- [Surface repository](https://github.com/mmore500/hstrat-surface-concept/tree/master)
